# **Installing Missing Libraries**

In [11]:
!pip install gymnasium
!pip install "gymnasium[atari,accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[classic-control]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


# Importing the libraries

In [12]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque , namedtuple


# Building A.I

# Creating the Architecture of the Nueral Network

In [13]:
class Network(nn.Module):
  def __init__(self,state_size,action_size,seed=42):
    super(Network,self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size,128)
    self.fc2 = nn.Linear(128,128)
    self.fc3 = nn.Linear(128,128)
    self.fc4 = nn.Linear(128,action_size)

  def forward(self,state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    X = self.fc3(x)
    x = F.relu(x)

    return self.fc4(x)


# Training The AI

# Setting up the environment

In [14]:
import gymnasium as gym

env = gym.make('CartPole-v1')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]


action_shape = env.action_space.shape
number_actions = env.action_space.n


# Initializing the hyperparameters

In [15]:
learning_rate = 7e-4
minibatch_size = 200
discount_factor = 0.999
replay_buffer_size = int(3e5)
interpolation_parameter = 1e-3

# Implementing Experience Replay

In [16]:
class ReplayMemory(object):
  def __init__(self,capacity):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []

  def push(self,event):
    self.memory.append(event)
    if len(self.memory)>self.capacity:
      del self.memory[0]

  def sample(self,batch_size):
    experiences = random.sample(self.memory,k=batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states , next_states , actions , rewards , dones


# Implementing the DQN Class

In [17]:
class Agent():
  def __init__(self,state_size,action_size):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = Network(state_size,action_size).to(self.device)
    self.target_qnetwork = Network(state_size,action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(),lr=learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0

  def step(self,states,actions,rewards,next_states,dones):
    self.memory.push((states,actions,rewards,next_states,dones))
    self.t_step = (self.t_step +1) % 4
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences,discount_factor)

  def act(self,state,epsilon=0.):
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random()>epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self,experiences,discount_factor):
    states,next_states,actions,rewards,dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    q_expected = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  def soft_update(self, local_model, target_model, interpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

# Initializing the DQN agent

In [18]:
agent = Agent(state_size,number_actions)

# Training the DQN agent

In [19]:
number_episodes = 5000
maximum_number_timesteps_per_episode = 500
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)


for episode in range(1, number_episodes + 1):
  state, _ = env.reset()
  score = 0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    reward = next_state[0] + 1  # reward shaping
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)
  print('\rEpisode {}\tAverage Score: {:}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 500.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: 21.985259739640387
Episode 200	Average Score: 15.510880373504842
Episode 300	Average Score: 13.890973984337906
Episode 400	Average Score: 11.956956088752559
Episode 500	Average Score: 13.628697000426564
Episode 600	Average Score: 18.219508850630117
Episode 700	Average Score: 53.88295670954729
Episode 800	Average Score: 51.48509303176115
Episode 900	Average Score: 50.31763643242931
Episode 1000	Average Score: 51.351732606070726
Episode 1100	Average Score: 52.638368130868386
Episode 1200	Average Score: 53.70233652528201
Episode 1300	Average Score: 55.06225647075011
Episode 1400	Average Score: 57.72396619961004
Episode 1500	Average Score: 60.29289575309813
Episode 1600	Average Score: 78.18646330191748
Episode 1700	Average Score: 94.14711984509347
Episode 1800	Average Score: 186.22433741888264
Episode 1900	Average Score: 259.1615429750125
Episode 2000	Average Score: 258.5459422231513
Episode 2100	Average Score: 231.70998946231674
Episode 2200	Average Score: 222.1

# Visualizing the Result

In [20]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
import os

def show_video_of_model(agent, env_name, max_frames=1000):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frame_number = 0

    while not done and frame_number < max_frames:
        frame = env.render()
        imageio.imwrite(f'frame_{frame_number}.png', frame)
        frame_number += 1
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)

    env.close()

    with imageio.get_writer('Cart-Pole.mp4', fps=30) as video_writer:
        for i in range(frame_number):
            frame = imageio.imread(f'frame_{i}.png')
            video_writer.append_data(frame)

    # Clean up the individual frame files
    for i in range(frame_number):
        os.remove(f'frame_{i}.png')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video_of_model(agent, 'CartPole-v1')
show_video()


<ipython-input-20-3a44809871f9>:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frame = imageio.imread(f'frame_{i}.png')
